# **Settings**

In [39]:
import os
import warnings
warnings.filterwarnings('ignore')                       # warning 출력 false

import numpy as np 
import pandas as pd
import re

# **1. Model Settings**

In [40]:
from ratsnlp.nlpbook.classification import ClassificationDeployArguments

# model setting
model_path = './kcbert3'

# arguments
args = ClassificationDeployArguments(
    pretrained_model_name = 'beomi/kcbert-base',
    downstream_model_dir = model_path,
    max_seq_length = 128
)


downstream_model_checkpoint_fpath: ./kcbert2\epoch=5-val_loss=0.01.ckpt


# **2. Tokenizer**

In [3]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained(
    args.pretrained_model_name,
    do_lower_case = False
)

# **3. Load Checkpoint**

In [41]:
# Load checkpoint
import torch
fine_tuned_model_ckpt = torch.load(
    args.downstream_model_checkpoint_fpath,
    map_location=torch.device('cpu')
)

# **4. Model Config**

In [42]:
from transformers import BertConfig, BertForSequenceClassification

# 모델 생성 및 초기화
from transformers import BertConfig
pretrained_model_config = BertConfig.from_pretrained(
    args.pretrained_model_name,
    num_labels = fine_tuned_model_ckpt['state_dict']['model.classifier.bias'].shape.numel()
)

model = BertForSequenceClassification(pretrained_model_config)

# **5. Model**

In [43]:
# 모델 불러오기
model.load_state_dict({k.replace('model.',''): v for k,v in fine_tuned_model_ckpt['state_dict'].items()})
model.eval()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30000, 768, padding_idx=0)
      (position_embeddings): Embedding(300, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

# 6. Inference

## **1) 감정 정보**

In [44]:
train_data = pd.read_csv('./data3/train_data.csv')
val_data = pd.read_csv('./data3/val_data.csv')
print(train_data.shape, val_data.shape)

(453965, 4) (56248, 4)


In [45]:
train_data['emotion'].unique()

for emotion in train_data['emotion'].unique():
    sub_emotion = train_data[train_data['emotion']==emotion]['sensitivity'].unique()
    print(f'{emotion}: {sub_emotion}')

Happy: ['고맙다' '굉장하다' '만족스럽다' '뭉클하다' '반갑다' '부유하다' '사랑스럽다' '영광스럽다' '자랑스럽다' '자유롭다'
 '즐겁다' '행복하다' '홀가분하다' '화목하다' '흐뭇하다']
Sad: ['가엾다' '괴롭다' '그립다' '보잘것없다' '불쌍하다' '불행하다' '뼈아프다' '서럽다' '시무룩하다' '쓸쓸하다' '아쉽다'
 '안타깝다' '억울하다']
Angry: ['경멸스럽다' '기막히다' '끔찍하다' '무례하다' '밉다' '배은망덕하다' '부당하다' '뻔뻔스럽다' '아니꼽다' '악랄하다'
 '억울하다' '언짢다' '짜증스럽다' '치욕스럽다' '한심하다']
Anxious: ['갑갑하다' '고통스럽다' '꺼림칙하다' '다급하다' '두렵다' '불확실하다' '수상하다' '심란하다' '아득하다' '약하다'
 '외롭다' '위험하다' '의심스럽다' '조마조마하다' '혼란하다']
Hurt: ['고통스럽다' '괴롭다' '냉정하다' '매정하다' '못되다' '무관심하다' '밉다' '불행하다' '뼈아프다' '서럽다' '섭섭하다'
 '슬프다' '쓰리다']
Embarrassed: ['갑작스럽다' '곤란하다' '급하다' '기막히다' '난데없다' '남사스럽다' '망하다' '버겁다' '부끄럽다' '아찔하다'
 '어리둥절하다' '엉뚱하다' '창피하다' '혼란하다']
Neutrality: ['긴밀하다' '녹녹하다' '담백하다' '대동소이하다' '둥그스름하다' '뚜렷하다' '얕다' '어렴풋하다' '엄밀하다' '엄중하다'
 '예사롭다' '자세하다']


In [46]:
import pickle
with open('./data2/encoder.pkl', 'rb') as f:
    encoder = pickle.load(f)
encoder.classes_

array(['Angry', 'Anxious', 'Embarrassed', 'Happy', 'Hurt', 'Neutrality',
       'Sad'], dtype=object)

## **2) 노래가사 감정 추론**

In [63]:
# 추론
def inference_fn(sentence):
    emotion_list = ['분노', '불안', '당황', '기쁨', '상처', '중립','슬픔']
    inputs = tokenizer(
        [sentence],
        max_length=args.max_seq_length,
        padding = 'max_length',
        truncation=True
    )

    with torch.no_grad():
        outputs = model(**{k: torch.tensor(v) for k,v in inputs.items()})
        prob = outputs.logits.softmax(dim=1)
        positive_prob = round(prob[0][1].item(),4)
        negative_prob = round(prob[0][0].item(),4)
        pred = torch.argmax(prob).item()
        
        return emotion_list[pred]
    
# 추론
def lyrics_preprocessing(lyrics):
    lyrics = re.sub('[^ㄱ-ㅣ가-힣]',' ',lyrics).split('  ')
    lyrics = [x .strip() for x in lyrics if x != '']
    lyrics = ' '.join(list(set(lyrics)))
    
    return lyrics

def inference_fn_lyrics(lyrics):
    emotion_list = ['분노', '불안', '당황', '기쁨', '상처', '중립','슬픔']
    
    sentence = lyrics_preprocessing(lyrics)
    if len(sentence) == 0:
        return 'X'
    
    inputs = tokenizer(
        [sentence],
        max_length=args.max_seq_length,
        padding = 'max_length',
        truncation=True
    )

    with torch.no_grad():
        outputs = model(**{k: torch.tensor(v) for k,v in inputs.items()})
        prob = outputs.logits.softmax(dim=1)
        pred = torch.argmax(prob).item()
        
        return emotion_list[pred]

In [64]:
song_data = pd.read_excel('./result_songDB.xlsx')
song_data

,분류키워드,가수,곡,장르,가사
0,기분전환,BIGBANG (빅뱅),거짓말,댄스,ye love is pain to all my brokenhearted people...
1,기분전환,소녀시대 (GIRLS' GENERATION),Gee,댄스,Uh Huh Listen Boy My First Love Story My Angel...
2,기분전환,원더걸스,Tell me (Sampling From 'Two Of Hearts'),댄스,너도 날 좋아할 줄은 몰랐어 어쩌면 좋아 너무나 좋아 꿈만 같아서 나 내 자신을 자...
3,기분전환,2NE1,내가 제일 잘 나가,댄스,내가 제일 잘 나가 내가 제일 잘 나가 내가 제일 잘 나가 내가 제일 잘 나가 제 ...
4,기분전환,미쓰에이,Bad Girl Good Girl,댄스,You don't know me You don't know me You don'...
...,...,...,...,...,...
4680,한강,죠지,somuch,R&B/Soul,Oh my god 어떻게 우리가 만나서 얼마나 행복해 I feel real happ...
4681,한강,죠지,하루종일,R&B/Soul,평소 같은 날들과 할 일 없는 나의 인생 그의 사이로 네가 들어와 줬음 싶어 할일없...
4682,한강,Red House,냉정과 열정 사이 (Feat. slchld),R&B/Soul,I just tell you mine but It ain’t 화장대 위에 먼지를 닦...
4683,한강,릴러말즈 (Leellamarz),Number (Feat. BIG Naughty (서동현)),R&B/Soul,오래 기다렸니 나도 기다렸는데 물어보고 싶은 게 300개가 넘네 그러니까 언제든 C...


In [ ]:
song_data['감정'] = song_data['가사'].apply(lambda x: inference_fn_lyrics(x))
song_data = song_data[song_data['감정']!='X']
print(song_data.shape)
song_data

In [ ]:
# 데이터 저장
song_data.to_csv('./songDB_add3.csv',index=False)

In [62]:
# 랜덤으로 확인하기
n = 20
rand_idx = np.random.randint(0,song_data.shape[0],n)

for idx in rand_idx:
    singer = song_data.iloc[idx,2]
    song = song_data.iloc[idx,1]
    lyrics = song_data.iloc[idx,4]
    emotion = inference_fn_lyrics(lyrics)
    print(f'[{singer}-{song}]의 감정은 {emotion}입니다.')

828
[경북 경산시-김승민]의 감정은 슬픔입니다.
0
[Changes-Hayd]의 감정은 분노입니다.
240
[ONCE AGAIN-윈터 (WINTER), 닝닝 (NINGNING)]의 감정은 슬픔입니다.
707
[할 수 있어-엔알지 (NRG)]의 감정은 상처입니다.
532
[사랑할 수 없는 너에게-도영 (DOYOUNG)]의 감정은 슬픔입니다.
272
[1,2,3,4 (원,투,쓰리,포)-이하이]의 감정은 슬픔입니다.
426
[삐삐-아이유]의 감정은 상처입니다.
471
[여자는 말 못하고, 남자는 모르는 것들-정동하]의 감정은 분노입니다.
363
[천애고독 (天涯孤獨)-OoOo (오넷)]의 감정은 슬픔입니다.
0
[So Good (Stripped)-Halsey]의 감정은 분노입니다.
0
[Straight Line-Kryga]의 감정은 분노입니다.
685
[오늘 취하면 (Feat. 창모) (Prod. SUGA)-SURAN (수란)]의 감정은 슬픔입니다.
782
[남자 없이 잘 살아-미쓰에이]의 감정은 상처입니다.
447
[나 혼자서-티파니 (TIFFANY)]의 감정은 슬픔입니다.
400
[감사-김동률]의 감정은 기쁨입니다.
525
[마법소녀 (魔法少女)-오렌지 캬라멜]의 감정은 기쁨입니다.
515
[가슴소리-THERAY]의 감정은 상처입니다.
503
[넌 나의 우주-스무살]의 감정은 기쁨입니다.
0
[Hate Everything-지소울 (GSoul)]의 감정은 분노입니다.
370
[The Way U Are-동방신기 (TVXQ!)]의 감정은 분노입니다.


In [50]:
inference_fn('나랑 다이소 갈 사람')#??????????????????

'슬픔'